In [22]:
import numpy as np
from copy import deepcopy

In [ ]:
class Route(object):
    def __init__(self, route):
        self.route = route
        self.city_count = len(route)
        self.route_length = TSP.get_length(route)
        self.create_edges()

    def create_edges(self):
        self.edges = set()
        for i in range(self.city_count):
            edge = tuple(sorted([self.route[i-1], self.route[i]]))
            self.edges.add(edge)
            
    def __getitem__(self, idx):
        return self.route[idx]

    def __contains__(self, edge):
        return edge in self.edges

    def get_index(self, node):
        return self.route.index(node)

    # Get previous and next node of current
    def neighbors(self, node):
        idx = self.route.index(node)
        prev = idx - 1
        next_ = idx + 1
        if next_ == self.city_count:
            next_ = 0

        return (self.route[prev], self.route[next_])

    def prev(self, idx):
        return self.route[idx-1]
    def next_(self, idx):
        return self.route[idx+1]

    def generate(self, edges_remove, edges_add):
        edges = (self.edges - edges_remove) | edges_add
        
        if len(edges) < self.city_count:
            return False, list()

        node = 0
        successors = dict()
        # Build the list of successors
        while len(edges) > 0:
            for i, j in edges:
                if i == node:
                    successors[node] = j
                    node = j
                    break
                elif j == node:
                    successors[node] = i
                    node = i
                    break
            edges.remove((i, j))
        # Similarly, if not every node has a successor, this can not work
        if len(successors) < self.size:
            return False, list()

        _next_ = successors[0]
        new_route = [0]
        seen = set(new_route)

        # If we already encountered a node it means we have a loop
        while _next_ not in seen:
            seen.add(_next_)
            new_tour.append(_next_)
            _next_ = successors[_next_]

        # If we visited all nodes without a loop we have a tour
        return len(new_route) == self.city_count, new_route

In [67]:
class TSP(object):
    # Global variables
    node_coordinates = np.nan
    distance_matrix = np.nan
    routes = dict()
    
    def __init__(self, node_coord, dist_matrix, route):
        TSP.node_coordinates = node_coord
        TSP.distance_matrix = dist_matrix
        self.Route = Route(route)
        # Random solution to start with
        self.start_path = np.random.permutation(len(node_coord))
        self.start_path_length = self.get_length(self.start_path)
        # Current heuristic optimization path
        self.path = self.start_path
        self.path_length = self.start_path_length
    
    # Compute path length
    def get_length(self, path):
        length = TSP.distance_matrix[path[0], path[-1]]
        for idx in range(len(path)-1):
            cost += TSP.distance_matrix(path[idx], path[idx+1])
        return length
    
    # Remember encountered paths
    def remember(self, path, length):
        self.path = path
        self.path_length = length
        TSP.routes[tuple(sorted(path))] = dict(path=path, length=length)

In [68]:
class LinKernighanHelsgaun(object):
    def __init__(self, Tsp, Route):
        self.solutions = set()
        self.neighbors  = dict()
        self.Tsp = Tsp
        self.Route = Route
    
    def optimize(self):
        # Check if route exists before optimization 
        route = tuple(sorted(self.Tsp.path))
        if route in TSP.routes:
            self.Tsp.path = TSP.routes[route]["path"]
            self.Tsp.path_length = TSP.routes[route]["length"]
            return self.Tsp.path, self.Tsp.path_length
    
        for i in self.Tsp.path:
            self.neighbors[i] = list()

            for j, dist in enumerate(self.Route.edges[i]):
                if dist > 0 and j in self.Tsp.path:
                    self.neighbours[i].append(j)

        # Restart the loop each time we find an improving candidate
        better = True
        while better:
            better = self.improve()
            # Paths always begin at 0 so this should manage to find duplicate solutions
            self.solutions.add(tuple(self.Tsp.path))

        self.Tsp.remember(self.Tsp.path, self.Tsp.path_length)
        
        return self.Tsp.path, self.Tsp.path_length

    def closest(self, node_relink, route, profit_gain, edges_remove, edges_add):
        """
        Find the closest neighbours of a node ordered by potential gain.  As a
        side-effect, also compute the partial improvement of joining a node.

        Parameters:

            - t2i: node to relink from

            - tour: current tour to optimise

            - gain: current gain

            - broken: set of edges to remove (X)

            - joined: set of edges to join (Y)

        Return: sorted list of neighbours based on potential improvement with
        next omission
        """
        neighbors = dict()

        # Create the neighbors of node_relink
        for node in self.neighbors[node_relink]:
            edge_add_i = sorted([node_relink, node])
            gain_i = profit_gain - TSP.get_length(node_relink, node)
            
            # Check if edge to add has profit gain, is not present in edges to remove
            # and is not present in the route
            if gain_i <= 0 or edge_add_i in edges_add or edge_add_i in self.Tsp:
                continue

            for _next_ in self.Tsp.neighbors(node):
                edge_remove_i = sorted([node, succ])

                # TODO verify it is enough, but we do check if the tour is
                # valid first thing in `chooseX` so this should be sufficient
                #
                # Check that "x_i+1 exists"
                if edge_remove_i not in edges_remove and edge_remove_i not in edges_add:
                    gain_diff = TSP.get_length(node, _next_) - TSP.get_length(node_relink, node)

                    if node in neighbours and diff > neighbours[node][0]:
                        neighbours[node][0] = gain_diff
                    else:
                        neighbours[node] = [gain_diff, gain_i]
                        
        neighbors = sorted(neighbors.items(), key=lambda x: x[1][0], reverse=True)
        return neighbors

    def improve(self):
        """
        Start the LKH algorithm with the current tour.
        """
        route = Route(self.heuristic_path)

        # Find all valid 2-opt moves and try them
        for t1 in self.heuristic_path:
            around = tour.around(t1)

            for t2 in around:
                broken = set([makePair(t1, t2)])
                # Initial savings
                gain = TSP.get_length(t1, t2)

                close = self.closest(t2, tour, gain, broken, set())

                # Number of neighbours to try
                tries = 5

                for t3, (_, Gi) in close:
                    # Make sure that the new node is none of t_1's neighbours
                    # so it does not belong to the tour.
                    if t3 in around:
                        continue

                    joined = set([makePair(t2, t3)])

                    if self.chooseX(tour, t1, t3, Gi, broken, joined):
                        # Return to Step 2, that is the initial loop
                        return True
                    # Else try the other options

                    tries -= 1
                    # Explored enough nodes, change t_2
                    if tries == 0:
                        break

        return False

    def chooseX(self, tour, t1, last, gain, broken, joined):
        """
        Choose an edge to omit from the tour.

        Parameters:

            - tour: current tour to optimise

            - t1: starting node for the current k-opt

            - last: tail of the last edge added (t_2i-1)

            - gain: current gain (Gi)

            - broken: potential edges to remove (X)

            - joined: potential edges to add (Y)

        Return: whether we found an improved tour
        """
        if len(broken) == 4:
            pred, succ = tour.around(last)

            # Give priority to the longest edge for x_4
            if TSP.get_length(pred, last) > TSP.get_length(succ, last):
                around = [pred]
            else:
                around = [succ]
        else:
            around = tour.around(last)

        for t2i in around:
            xi = makePair(last, t2i)
            # Gain at current iteration
            Gi = gain + TSP.get_length(last, t2i)

            # Verify that X and Y are disjoint, though I also need to check
            # that we are not including an x_i again for some reason.
            if xi not in joined and xi not in broken:
                added = deepcopy(joined)
                removed = deepcopy(broken)

                removed.add(xi)
                added.add(makePair(t2i, t1))  # Try to relink the tour

                relink = Gi - TSP.dist(t2i, t1)
                is_tour, new_tour = tour.generate(removed, added)

                # The current solution does not form a valid tour
                if not is_tour and len(added) > 2:
                    continue

                # Stop the search if we come back to the same solution
                if str(new_tour) in self.solutions:
                    return False

                # Save the current solution if the tour is better, we need
                # `is_tour` again in the case where we have a non-sequential
                # exchange with i = 2
                if is_tour and relink > 0:
                    self.heuristic_path = new_tour
                    self.heuristic_cost -= relink
                    return True
                else:
                    # Pass on the newly "removed" edge but not the relink
                    choice = self.chooseY(tour, t1, t2i, Gi, removed, joined)

                    if len(broken) == 2 and choice:
                        return True
                    else:
                        # Single iteration for i > 2
                        return choice

        return False

    def chooseY(self, tour, t1, t2i, gain, broken, joined):
        """
        Choose an edge to add to the new tour.

        Parameters:

            - tour: current tour to optimise

            - t1: starting node for the current k-opt

            - t2i: tail of the last edge removed (t_2i)

            - gain: current gain (Gi)

            - broken: potential edges to remove (X)

            - joined: potential edges to add (Y)

        Return: whether we found an improved tour
        """
        ordered = self.closest(t2i, tour, gain, broken, joined)

        if len(broken) == 2:
            # Check the five nearest neighbours when i = 2
            top = 5
        else:
            # Otherwise the closest only
            top = 1

        for node, (_, Gi) in ordered:
            yi = makePair(t2i, node)
            added = deepcopy(joined)
            added.add(yi)

            # Stop at the first improving tour
            if self.chooseX(tour, t1, node, Gi, broken, added):
                return True

            top -= 1
            # Tried enough options
            if top == 0:
                return False

        return False